## Credit Card Fraud Detection

In [22]:
# import required libraries
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from google.datalab.ml import TensorBoard

import shutil
import pandas as pd
import tensorflow as tf

print(tf.__version__)

1.8.0


In [23]:
# configure logging
tf.logging.set_verbosity(tf.logging.INFO)
pd.options.display.max_rows = 50
pd.options.display.float_format = '{:.1f}'.format

In [24]:
OUTDIR = './trained_model/v01'

In [25]:
# makes result reproducible
#np.random.seed(seed=1) 
#tf.random.set_random_seed(seed)

In [26]:
df_train = pd.read_csv('./data/preprocess/creditcard_header_train.csv')
df_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V9,V10,V11,V12,V14,V16,V17,V18,V19,V21,Amount,Class
0,77627.0,-7.1,2.8,-6.8,4.4,-5.5,-1.7,-6.5,-3.1,-6.3,2.4,-6.2,-5.6,-6.5,-11.4,-4.7,2.4,1.3,106.9,1
1,59011.0,-2.3,-3.3,-3.5,3.2,-2.8,-0.2,-0.9,-0.8,-4.4,3.1,-2.7,-5.6,-2.5,-4.5,-0.9,0.6,1.2,1389.6,1
2,8757.0,-1.9,3.4,-4.5,2.8,-2.1,-2.3,-4.3,-1.4,-7.0,6.3,-8.7,-7.4,-3.7,-6.3,-1.2,0.4,0.7,1.0,1
3,125658.0,0.2,3.0,-3.4,4.0,3.8,0.2,1.1,-2.4,-2.3,1.8,-2.0,-7.1,2.1,6.4,1.6,-3.7,0.5,0.0,1
4,64093.0,-6.1,2.9,-5.6,3.3,-5.3,-0.6,-4.5,-1.6,-5.1,4.0,-8.1,-8.7,-6.2,-11.0,-4.2,1.8,0.9,294.9,1


In [27]:
df_train.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V9,V10,V11,V12,V14,V16,V17,V18,V19,V21,Amount,Class
count,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0,227846.0
mean,94841.4,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,88.1,0.0
std,47493.0,2.0,1.7,1.5,1.4,1.4,1.3,1.2,1.1,1.1,1.0,1.0,1.0,0.9,0.9,0.8,0.8,0.7,243.4,0.0
min,0.0,-56.4,-72.7,-33.7,-5.7,-42.1,-26.2,-43.6,-13.4,-24.6,-4.6,-18.7,-19.2,-14.1,-25.2,-9.5,-7.2,-34.8,0.0,0.0
25%,54191.0,-0.9,-0.6,-0.9,-0.8,-0.7,-0.8,-0.6,-0.6,-0.5,-0.8,-0.4,-0.4,-0.5,-0.5,-0.5,-0.5,-0.2,5.6,0.0
50%,84748.5,0.0,0.1,0.2,-0.0,-0.1,-0.3,0.0,-0.1,-0.1,-0.0,0.1,0.1,0.1,-0.1,-0.0,0.0,-0.0,22.0,0.0
75%,139372.8,1.3,0.8,1.0,0.7,0.6,0.4,0.6,0.6,0.5,0.7,0.6,0.5,0.5,0.4,0.5,0.5,0.2,77.2,0.0
max,172792.0,2.5,22.1,4.2,16.9,34.8,23.9,44.1,10.4,15.2,12.0,7.8,10.5,7.1,9.2,5.0,5.6,27.2,19656.5,1.0


In [28]:
df_test = pd.read_csv('./data/preprocess/creditcard_header_test.csv')
df_test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V9,V10,V11,V12,V14,V16,V17,V18,V19,V21,Amount,Class
0,1.0,-1.0,-0.2,1.8,-0.9,-0.0,1.2,0.2,-1.4,-0.1,-0.2,0.2,-0.3,-1.1,-0.7,2.0,-1.2,-0.1,123.5,0
1,7.0,-0.9,0.3,-0.1,-0.3,2.7,3.7,0.4,-0.4,-0.4,-0.7,-0.1,0.1,-0.2,-0.5,0.1,0.6,-0.1,93.2,0
2,10.0,0.4,0.6,-0.9,-0.1,2.9,3.3,0.5,-0.6,0.3,-0.3,-0.3,0.4,-0.1,-0.8,0.4,0.7,0.0,10.0,0
3,13.0,-0.4,0.9,0.9,-0.7,0.9,-0.1,0.7,-0.7,-0.7,0.3,0.3,-0.3,1.1,-0.9,0.7,0.0,-0.2,0.9,0
4,15.0,1.5,-1.0,0.5,-1.4,-1.6,-0.7,-1.1,-2.0,1.6,1.1,-0.6,0.1,-0.2,0.3,0.6,0.1,-0.2,5.0,0


In [29]:
df_test.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V9,V10,V11,V12,V14,V16,V17,V18,V19,V21,Amount,Class
count,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0,56961.0
mean,94703.5,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,89.5,0.0
std,47469.0,2.0,1.6,1.5,1.4,1.5,1.4,1.3,1.1,1.1,1.0,1.0,1.0,0.9,0.8,0.8,0.8,0.7,275.4,0.0
min,1.0,-37.6,-42.2,-48.3,-5.6,-113.7,-13.6,-31.2,-9.5,-22.2,-4.8,-16.1,-18.4,-13.3,-22.9,-9.3,-4.5,-22.9,0.0,0.0
25%,54248.0,-0.9,-0.6,-0.9,-0.8,-0.7,-0.8,-0.6,-0.6,-0.5,-0.8,-0.4,-0.4,-0.5,-0.5,-0.5,-0.5,-0.2,5.7,0.0
50%,84578.0,0.0,0.1,0.2,-0.0,-0.0,-0.3,0.0,-0.1,-0.1,-0.0,0.1,0.0,0.1,-0.1,-0.0,-0.0,-0.0,22.0,0.0
75%,139106.0,1.3,0.8,1.0,0.7,0.6,0.4,0.6,0.6,0.4,0.7,0.6,0.5,0.5,0.4,0.5,0.5,0.2,77.1,0.0
max,172787.0,2.4,16.5,9.4,15.3,19.2,73.3,120.6,15.6,23.7,10.1,4.5,7.8,17.3,9.3,4.3,5.5,22.6,25691.2,1.0


In [30]:
INPUT_COLUMNS = [
  tf.feature_column.numeric_column("Time", dtype=tf.float32),
  tf.feature_column.numeric_column("V1", dtype=tf.float32),
  tf.feature_column.numeric_column("V2", dtype=tf.float32),
  tf.feature_column.numeric_column("V3", dtype=tf.float32),
  tf.feature_column.numeric_column("V4", dtype=tf.float32),
  tf.feature_column.numeric_column("V5", dtype=tf.float32),
  tf.feature_column.numeric_column("V6", dtype=tf.float32),
  tf.feature_column.numeric_column("V7", dtype=tf.float32),
  tf.feature_column.numeric_column("V9", dtype=tf.float32),
  tf.feature_column.numeric_column("V10", dtype=tf.float32),
  tf.feature_column.numeric_column("V11", dtype=tf.float32),
  tf.feature_column.numeric_column("V12", dtype=tf.float32),
  tf.feature_column.numeric_column("V14", dtype=tf.float32),
  tf.feature_column.numeric_column("V16", dtype=tf.float32),
  tf.feature_column.numeric_column("V17", dtype=tf.float32),
  tf.feature_column.numeric_column("V18", dtype=tf.float32),
  tf.feature_column.numeric_column("V19", dtype=tf.float32),
  tf.feature_column.numeric_column("V21", dtype=tf.float32),
  tf.feature_column.numeric_column("Amount", dtype=tf.float32)
]

In [31]:
ADDITIONAL_COLUMNS = [
  tf.feature_column.numeric_column("V1_", dtype=tf.float32),
  tf.feature_column.numeric_column("V2_", dtype=tf.float32),
  tf.feature_column.numeric_column("V3_", dtype=tf.float32),
  tf.feature_column.numeric_column("V4_", dtype=tf.float32),
  tf.feature_column.numeric_column("V5_", dtype=tf.float32),
  tf.feature_column.numeric_column("V6_", dtype=tf.float32),
  tf.feature_column.numeric_column("V7_", dtype=tf.float32),
  tf.feature_column.numeric_column("V9_", dtype=tf.float32),
  tf.feature_column.numeric_column("V10_", dtype=tf.float32),
  tf.feature_column.numeric_column("V11_", dtype=tf.float32),
  tf.feature_column.numeric_column("V12_", dtype=tf.float32),
  tf.feature_column.numeric_column("V14_", dtype=tf.float32),
  tf.feature_column.numeric_column("V16_", dtype=tf.float32),
  tf.feature_column.numeric_column("V17_", dtype=tf.float32),
  tf.feature_column.numeric_column("V18_", dtype=tf.float32),
  tf.feature_column.numeric_column("V19_", dtype=tf.float32),
  tf.feature_column.numeric_column("V21_", dtype=tf.float32),
  tf.feature_column.numeric_column("Amount_max_fraud", dtype=tf.float32)
]

In [32]:
# Define your feature columns
def create_feature_cols():
  return INPUT_COLUMNS + ADDITIONAL_COLUMNS

In [33]:
def add_more_features(df):
  "Add additional features"
  # ToDo Add additional standardized columns
  df['Amount_max_fraud'] = 1
  df.loc[df.Amount <= 2125.87, 'Amount_max_fraud'] = 0
  df['V1_'] = df.V1.map(lambda x: 1 if x < -3 else 0)
  df['V2_'] = df.V2.map(lambda x: 1 if x > 2.5 else 0)
  df['V3_'] = df.V3.map(lambda x: 1 if x < -4 else 0)
  df['V4_'] = df.V4.map(lambda x: 1 if x > 2.5 else 0)
  df['V5_'] = df.V5.map(lambda x: 1 if x < -4.5 else 0)
  df['V6_'] = df.V6.map(lambda x: 1 if x < -2.5 else 0)
  df['V7_'] = df.V7.map(lambda x: 1 if x < -3 else 0)
  df['V9_'] = df.V9.map(lambda x: 1 if x < -2 else 0)
  df['V10_'] = df.V10.map(lambda x: 1 if x < -2.5 else 0)
  df['V11_'] = df.V11.map(lambda x: 1 if x > 2 else 0)
  df['V12_'] = df.V12.map(lambda x: 1 if x < -2 else 0)
  df['V14_'] = df.V14.map(lambda x: 1 if x < -2.5 else 0)
  df['V16_'] = df.V16.map(lambda x: 1 if x < -2 else 0)
  df['V17_'] = df.V17.map(lambda x: 1 if x < -2 else 0)
  df['V18_'] = df.V18.map(lambda x: 1 if x < -2 else 0)
  df['V19_'] = df.V19.map(lambda x: 1 if x > 1.5 else 0)
  df['V21_'] = df.V21.map(lambda x: 1 if x > 0.6 else 0)
  return df

In [34]:
def transform_feature_cols(df):
  """
  Transform each feature in features so that it has a mean of 0 and standard deviation of 1; 
  This helps with training the neural network.
  """
  features = df.columns.values
  for feature in features:
    if feature != 'Class':
      mean, std = df[feature].mean(), df[feature].std()
      df.loc[:, feature] = (df[feature] - mean) / std
  return df

In [35]:
def train_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = transform_feature_cols(add_more_features(df)),
    y = df['Class'],
    batch_size = 256,
    num_epochs = None,
    shuffle = True,
    queue_capacity = 1000,
    num_threads = 1
  )

In [36]:
def eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = transform_feature_cols(add_more_features(df)),
    y = df['Class'],
    batch_size = 128,
    num_epochs = 1,
    shuffle = False,
    num_threads = 1
  )

In [37]:
def serving_input_fn():
  """Servers prediction service"""
  feature_placeholders = {
      column.name: tf.placeholder(tf.float32, [None]) for column in INPUT_COLUMNS
  }
  features = add_more_features(pd.DataFrame.from_dict(feature_placeholders.copy()))
  return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [38]:
# ToDo impelement for custom estimator
def dnn_model(img, mode, hparams):
  pass

In [39]:
# ToDo impelement for custom estimator
def transaction_classifier(features, labels, mode, params):
  """Transaction classifier either fraud or normal"""
  pass

In [40]:
def train_and_evaluate(output_dir, num_train_steps):
  """Training and evalucation function"""
  EVAL_INTERVAL = 10
  feature_columns = create_feature_cols()
  estimator = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[37, 18, 9],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=2,
    dropout=0.1,
    model_dir=output_dir)
    
  train_spec=tf.estimator.TrainSpec(input_fn=train_input_fn(df_train), max_steps=num_train_steps)
  # ToDo Fix issue of add engg feature with panda read input functions
  # exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
  # eval_spec=tf.estimator.EvalSpec(input_fn=eval_input_fn(df_test), throttle_secs=EVAL_INTERVAL, exporters=exporter)
  eval_spec=tf.estimator.EvalSpec(input_fn=eval_input_fn(df_test), throttle_secs=EVAL_INTERVAL)
  
  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [45]:
# Launch tensorboard
TensorBoard().start(OUTDIR)

TensorBoard was started successfully with pid 44736. Click here to access it.

44736

In [42]:
# Run the model
shutil.rmtree(OUTDIR, ignore_errors = True)
train_and_evaluate(OUTDIR, 10000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': None, '_task_id': 0, '_is_chief': True, '_master': '', '_save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f844b5d3eb8>, '_global_id_in_cluster': 0, '_service': None, '_task_type': 'worker', '_train_distribute': None, '_model_dir': './trained_model', '_save_summary_steps': 100, '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_session_config': None, '_keep_checkpoint_max': 5, '_evaluation_master': '', '_save_checkpoints_steps': None, '_log_step_count_steps': 100}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 10 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorf

INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-11-14-17:07:38
INFO:tensorflow:Saving dict for global step 2490: accuracy = 0.9976562, accuracy_baseline = 0.9976562, auc = 0.63564605, auc_precision_recall = 0.024262588, average_loss = 0.015734455, global_step = 2490, label/mean = 0.00234375, loss = 2.0140102, precision = 0.0, prediction/mean = 0.001236022, recall = 0.0
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-2490
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2491 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 1.2969297, step = 2491
INFO:tensorflow:global_step/sec: 78.0226
INFO:tensorf

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-4979
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4980 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 0.69540906, step = 4980
INFO:tensorflow:global_step/sec: 79.031
INFO:tensorflow:loss = 0.2931092, step = 5080 (1.268 sec)
INFO:tensorflow:global_step/sec: 103.014
INFO:tensorflow:loss = 0.52221864, step = 5180 (0.971 sec)
INFO:tensorflow:global_step/sec: 99.7457
INFO:tensorflow:loss = 264.16275, step = 5280 (1.002 sec)
INFO:tensorflow:global_step/sec: 98.9494
INFO:tensorflow:loss = 0.5230064, step = 5380 (1.011 sec)
INFO:tensorflow:global_step/sec: 102.189
INFO:tensorflow:loss = 0.7583715, step = 5480 (0.978 sec)
INFO:tensorflow:global_step/sec: 102.304
INFO:tensorflow:loss = 0.7422

INFO:tensorflow:Saving checkpoints for 7455 into ./trained_model/model.ckpt.
INFO:tensorflow:loss = 0.2238938, step = 7455
INFO:tensorflow:global_step/sec: 78.4729
INFO:tensorflow:loss = 0.21473032, step = 7555 (1.276 sec)
INFO:tensorflow:global_step/sec: 97.4669
INFO:tensorflow:loss = 0.24970877, step = 7655 (1.026 sec)
INFO:tensorflow:global_step/sec: 94.3354
INFO:tensorflow:loss = 0.22389895, step = 7755 (1.060 sec)
INFO:tensorflow:global_step/sec: 95.1595
INFO:tensorflow:loss = 0.64661056, step = 7855 (1.051 sec)
INFO:tensorflow:global_step/sec: 96.9187
INFO:tensorflow:loss = 0.32609266, step = 7955 (1.032 sec)
INFO:tensorflow:global_step/sec: 93.1266
INFO:tensorflow:loss = 0.17545739, step = 8055 (1.074 sec)
INFO:tensorflow:global_step/sec: 96.5055
INFO:tensorflow:loss = 0.061434, step = 8155 (1.036 sec)
INFO:tensorflow:global_step/sec: 99.2667
INFO:tensorflow:loss = 0.12489021, step = 8255 (1.010 sec)
INFO:tensorflow:Saving checkpoints for 8267 into ./trained_model/model.ckpt.
IN

INFO:tensorflow:Loss for final step: 0.08473517.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-14-17:09:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./trained_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2018-11-14-17:09:54
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.9990625, accuracy_baseline = 0.9976562, auc = 0.93274224, auc_precision_recall = 0.7024772, average_loss = 0.004468976, global_step = 10000,

In [44]:
#for pid in TensorBoard.list()['pid']:
#  TensorBoard().stop(pid)
#  print('Stopped TensorBoard with pid {}'.format(pid))

<pre>
# Copyright 2018 Atos. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>